# Phase Picking of Denoised DAS

In [ ]:
import gc
import h5py
import torch
import numpy as np
import torch.nn as nn
from das_util import try_gpu
from matplotlib import pyplot as plt
from numpy.random import default_rng
from scipy.signal import filtfilt, butter
from torch.utils.data import DataLoader
from das_denoise_models import unet, dataflow, datalabel
from das_denoise_training import train_augmentation
from sklearn.model_selection import train_test_split

## 1. Read and pre-filter the benchmark data

Use the __seismo__ ipykernel on **Siletzia** for two reasons:

1. It has GPUs

2. It stores AK-DAS data

In [ ]:
! du -sh /fd1/QibinShi_data/akdas/qibin_data/KKFLStill2023*

In [ ]:
### Read (change the file to the latest if needed)
data_terra = '/fd1/QibinShi_data/akdas/qibin_data/TERRAtill2023_12_12.hdf5'
data_kkfls = '/fd1/QibinShi_data/akdas/qibin_data/KKFLStill2023_12_12.hdf5'
with h5py.File(data_terra, 'r') as f:
    quake1 = f['raw_quake'][:]
with h5py.File(data_kkfls, 'r') as f:
    quake2 = f['raw_quake'][:]

In [ ]:
### Filter
sample_rate = 25
delta_space = 10
rawdata = np.append(quake2[:,500:2000,:], quake1[:,500:2000,:], axis=0)
b, a = butter(4, (0.5, 12), fs=sample_rate, btype='bandpass')
filt = filtfilt(b, a, rawdata, axis=2)
rawdata = filt / np.std(filt, axis=(1,2), keepdims=True)  ## Rawdata w.r.t. Denoised 

In [ ]:
from obspy import read_events

qml = '/fd1/QibinShi_data/akdas/qibin_data/ak_July1_3.xml'

cat = read_events(qml)

evt = cat[48%48]
mag = evt.magnitudes[0].mag
lat = evt.origins[0].longitude
lon = evt.origins[0].latitude
evt = evt.origins[0].time

print(mag, lat, lon, evt)

In [ ]:
print(cat[94%48])

## 2. Load models trained in different ways

In [ ]:
""" Initialize the U-net model """
devc = try_gpu(i=1)

model_1 = unet(1, 16, 1024, factors=(5, 3, 2, 2), use_att=False)
model_1 = nn.DataParallel(model_1, device_ids=[1,2,3])
model_1.to(devc)
model_1.load_state_dict(torch.load('models/checkpoint_noatt_LRdecays0.8_mask0.5_raw2raw_chmax4500.pt'))  # raw2raw
model_1.eval() 

## 3. Denoise to compare

In [ ]:
X=rawdata[:100,:,:].astype(np.float32)

del rawdata
gc.collect()

X = torch.from_numpy(X).to(devc)
### denoise
with torch.no_grad():
    ### raw2raw
    oneDenoise_1 = model_1(X)
    mulDenoise_1 = model_1(oneDenoise_1)
    mulDenoise_1 = model_1(mulDenoise_1)
    mulDenoise_1 = model_1(mulDenoise_1)

### convert back to numpy, trim edges
rawdata = X.to('cpu').numpy()
oneDenoise_1 = oneDenoise_1.to('cpu').numpy()
mulDenoise_1 = mulDenoise_1.to('cpu').numpy()

### save denoised data to file
# with h5py.File('/fd1/QibinShi_data/akdas/qibin_data/' + 'raw_and_one_mul_denoise.hdf5', 'w') as f:
#     f.create_dataset("raw", data=rawdata)
#     f.create_dataset("raw2raw_oneDenoise", data=oneDenoise_1)
#     f.create_dataset("fk2fk_oneDenoise", data=oneDenoise_2)
#     f.create_dataset("raw2fk_oneDenoise", data=oneDenoise_3)
#     f.create_dataset("raw2raw_mulDenoise", data=mulDenoise_1)
#     f.create_dataset("fk2fk_mulDenoise", data=mulDenoise_2)
#     f.create_dataset("raw2fk_mulDenoise", data=mulDenoise_3)

In [ ]:
def process_3d_array(arr, len1=1500, len2=1500):
    """convert to numpy array"""
    arr = np.array(arr)
    
    """Ensure the array has at least len1 rows and len2 columns"""
    slices, rows, cols = arr.shape
    arr = arr[:, :min(rows, len1), :min(cols, len2)]
    
    """Pad zeros if it has fewer than len1 rows or len2 columns"""
    if rows < len1 or cols < len2:
        padding_rows = max(len1 - rows, 0)
        padding_cols = max(len2 - cols, 0)
        arr = np.pad(arr, ((0, 0), (0, padding_rows), (0, padding_cols)), 'constant')
    
    return arr

def vizRawDenoise(in_data, oneDenoise, mulDenoise, sample_rate=25, dchan=10, index=[0,1], model="MAE"):
    """
    in_data, oneDenoise, mulDenoise: 3D -- [event, channel, time]
    index: list, subset of the events
    model: string, descriptions about the model
    """
    len1, len2 = oneDenoise[0].shape[0], oneDenoise[0].shape[1]
    x, y = np.arange(len2), np.arange(len1)
    rawdata = process_3d_array(in_data, len1=len1, len2=len2)

    for j in index:
        bound = np.median(np.fabs(in_data[j]))*2
        fig, ax = plt.subplots(1, 3, figsize=(18, 6), constrained_layout=True)

        img=ax[0].pcolormesh(x, y, rawdata[j], shading='auto', vmin=-bound, vmax=bound, cmap=plt.cm.get_cmap('RdBu'))
        ax[1].pcolormesh(x, y, oneDenoise[j], shading='auto',  vmin=-bound, vmax=bound, cmap=plt.cm.get_cmap('RdBu'))
        ax[2].pcolormesh(x, y, mulDenoise[j], shading='auto', vmin=-bound, vmax=bound, cmap=plt.cm.get_cmap('RdBu'))

        ax[0].set_title("Raw data #"+str(j), fontsize=24)
        ax[1].set_title(model+" 1-time denoised", fontsize=24)
        ax[2].set_title(model+" multi-time denoised", fontsize=24)
        ax[0].set_ylabel('Distance (km)', fontsize=24)

        plt.colorbar(img, ax=ax[2])

        for i in range(3):
            ax[i].set_xlabel('Time (s)', fontsize=24)
            ax[i].set_xticks(np.arange(0, 250*(len2//250), 250)) 
            ax[i].set_xticklabels(np.arange(0, 250*(len2//250)/sample_rate, 250/sample_rate).astype(int))
            ax[i].set_yticks(np.arange(0, 200*(len1//200), 200))
            ax[i].set_yticklabels((np.arange(0, dchan*200*(len1//200), 200*dchan)/1000).astype(int))
            
vizRawDenoise(rawdata, oneDenoise_1, mulDenoise_1, index=range(50), model="raw-raw")

# Phase picking

### Read the denoised data (if you don't run the cells above)

In [ ]:
import sys
sys.path.append("../src/denoiser/")
sys.path.append("../src/ensemble_picker/")
sys.path.append("models/")
import os
import gc
import h5py
import torch
import matplotlib
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.multiprocessing as mp
from tqdm import tqdm
from matplotlib import pyplot as plt
from numpy.random import default_rng
from scipy.signal import filtfilt, butter
from torch.utils.data import DataLoader
from scipy.interpolate import interp1d
from collections import OrderedDict
from obspy import read_events

# This package
from das_denoise_models import unet, dataflow, datalabel
from das_denoise_training import train_augmentation
from das_util import try_gpu

# seisbench
import seisbench.models as sbm

# some ELEP functions
# from mbf_elep_func import apply_elep


# import os
# devc = torch.cuda.current_device()
# os.environ['CUDA_VISIBLE_DEVICES'] = str(devc)

matplotlib.rcParams.update({'font.size': 20})

In [ ]:
# ML picker parameters
paras_semblance = {'dt':0.01, 
                   'semblance_order':2, 
                   'window_flag':True, 
                   'semblance_win':0.5, 
                   'weight_flag':'max'}

p_thrd, s_thrd = 0.01, 0.05
fqmin = 0.1
fqmax = 5

dt = 0.01; fs = 100
nfqs = 10
nt = 6000; nc = 3
fq_list = [1]  # make_LogFq(fqmin, fqmax, dt, nfqs)
coeff_HP, coeff_LP = [1,1]  # rec_filter_coeff(fq_list, dt)
MBF_paras = {'f_min':fqmin, 
             'f_max':fqmax,
             'nfqs':nfqs, 
             'frequencies':fq_list, 
             'CN_HP':coeff_HP, 
             'CN_LP':coeff_LP,
             'dt':dt, 
             'fs':fs, 
             'nt':nt, 
             'nc':nc, 
             'npoles': 2}

In [ ]:
# download models
devcc = torch.device(f'cuda:{1}')
pretrain_list = ["ethz","instance","scedc","stead","geofon","neic"]
# pn_pnw_model = sbm.EQTransformer.from_pretrained('original')
pn_ethz_model = sbm.EQTransformer.from_pretrained("ethz")
pn_instance_model = sbm.EQTransformer.from_pretrained("instance")
pn_scedc_model = sbm.EQTransformer.from_pretrained("scedc")
pn_stead_model = sbm.EQTransformer.from_pretrained("stead")
pn_geofon_model = sbm.EQTransformer.from_pretrained("geofon")
pn_neic_model = sbm.EQTransformer.from_pretrained("neic")

list_models = [pn_ethz_model,pn_scedc_model,pn_neic_model,pn_geofon_model,pn_stead_model,pn_instance_model]

In [ ]:
pn_ethz_model.to(devcc);
pn_scedc_model.to(devcc);
pn_neic_model.to(devcc);
pn_geofon_model.to(devcc);
pn_stead_model.to(devcc);
pn_instance_model.to(devcc);

In [ ]:
# interpolate
interpolation_function = interp1d(np.linspace(0, 1, 1500), rawdata[0:100,:,:], axis=-1, kind='linear')
interpolated_image = interpolation_function(np.linspace(0, 1, 6000))
interpolation_function = interp1d(np.linspace(0, 1, 1500), oneDenoise_1[0:100,:,:], axis=-1, kind='linear')
interpolated_onedenoised = interpolation_function(np.linspace(0, 1, 6000))
interpolation_function = interp1d(np.linspace(0, 1, 1500), mulDenoise_1[0:100,:,:], axis=-1, kind='linear')
interpolated_muldenoised = interpolation_function(np.linspace(0, 1, 6000))

In [ ]:
import time as time
from joblib import Parallel, delayed
from ELEP.elep.ensemble_coherence import ensemble_semblance 


def apply_elep(DAS_data, list_models, MBF_paras, paras_semblance, \
              thr=0.6,device=devc):
    
    """"
    This function takes a array of stream, a list of ML models and 
    apply these models to the data, predict phase picks, and
    return an array of picks .
    DAS_data: NDArray of DAS data: [channel,time stamp - 6000]
    """
    
    twin = 6000
    nsta = DAS_data.shape[0]
    bigS = np.zeros(shape=(DAS_data.shape[0], 3, DAS_data.shape[1]))
    for i in range(nsta):
        bigS[i,0,:] = DAS_data[i,:]
        bigS[i,1,:] = DAS_data[i,:]
        bigS[i,2,:] = DAS_data[i,:]

    # allocating memory for the ensemble predictions
    batch_pred_P = np.zeros(shape=(len(list_models),nsta,twin)) 
    batch_pred_S = np.zeros(shape=(len(list_models),nsta,twin))
        
    ######### Broadband workflow ################
    crap2 = bigS.copy()
    crap2 -= np.mean(crap2, axis=-1, keepdims= True) # demean data
    # original use std norm
    data_std = crap2 / (np.std(crap2) + 1e-7)
    # could use max data
    mmax = np.max(np.abs(crap2), axis=-1, keepdims=True)
    data_max = np.divide(crap2 , mmax,out=np.zeros_like(crap2), where=mmax!=0)
    del bigS
    
    # data to tensor
    data_tt = torch.from_numpy(data_max).to(device, dtype=torch.float64)
    
    t0=time.time()
    for ii, imodel in enumerate(list_models):
        imodel.to(device)
        imodel=imodel.double()
        imodel.eval()
        with torch.no_grad():
            batch_pred_P[ii, :, :] = imodel(data_tt)[1].cpu().numpy()[:, :]
            batch_pred_S[ii, :, :] = imodel(data_tt)[2].cpu().numpy()[:, :]
    
    print(f"Picks predicted in broadband workflow on the GPU in {time.time()-t0} seconds")
    
    smb_peak = np.zeros([nsta,2,2], dtype = np.float32)

    sfs = MBF_paras["fs"]
    istart = 0 

    def process_p(ista,paras_semblance,batch_pred,istart):
        crap = ensemble_semblance(batch_pred[:, ista, :], paras_semblance)
        imax = np.argmax(crap[istart:])
        prob = crap[istart+imax]
        smb_peak=0
        if crap[imax+istart] > thr:
            smb_peak = float((imax)/sfs)+istart/sfs 
            
        return smb_peak, prob

    smb_peak[:,0,:] = np.array(Parallel(n_jobs=100)(delayed(process_p)(ista,paras_semblance,batch_pred_P,istart) for ista in range(nsta)))
    smb_peak[:,1,:] = np.array(Parallel(n_jobs=100)(delayed(process_p)(ista,paras_semblance,batch_pred_S,istart) for ista in range(nsta)))
    
    return smb_peak

In [ ]:
tzoom=np.arange(-1,1,0.04)
tzoom.shape


In [ ]:
Nevent=96

devc = torch.device(f'cuda:{1}')
fs=100
wd=int(fs * 2)
ch_itv=10
dchan=10
tax=np.arange(1500)/25
tzoom=np.arange(-1,1,0.04)
thr=0.05

for i in np.arange(Nevent):
    
    evt = cat[i%48]
    mag = evt.magnitudes[0].mag
    lat = evt.origins[0].longitude
    lon = evt.origins[0].latitude
    
    # pick on the RAW data
    pickr=apply_elep(interpolated_image[i,::ch_itv,:], list_models, MBF_paras, paras_semblance,thr=thr,device=devc)
    # pick on the denoised data
    pick1=apply_elep(interpolated_onedenoised[i,::ch_itv,:], list_models, MBF_paras, paras_semblance,thr=thr,device=devc)
    pick2=apply_elep(interpolated_muldenoised[i,::ch_itv,:], list_models, MBF_paras, paras_semblance,thr=thr,device=devc) 

    colors = ['blue', 'green']
    labels = ['P BB', 'S BB']
    
    ######## Plotting
    cmap=plt.cm.get_cmap('RdBu')
    image = rawdata[i,::ch_itv,:]
    bound = np.median(np.fabs(image))*2
    x, y = (np.arange(image.shape[1])/25), (np.arange(image.shape[0])*ch_itv*dchan/1000)
    plt.figure(figsize=(36, 12), constrained_layout=True)
    plt.subplot(2, 6, 1) 
    plt.pcolormesh(x, y, image, shading='auto', vmin=-bound, vmax=bound, cmap=cmap)
#     plt.scatter(pickr[:,0,0],y, s=12,marker='o',c="b",alpha=0.3)
    plt.scatter(pickr[:,1,0],y, s=12,marker='o',c=colors[1],alpha=0.3)
    plt.title("M"+str(mag)+" lat. "+str(lat)+" lon. "+str(lon))
    plt.xlabel("Time since starttime (s)")
    plt.ylabel("Distance along cable (km)")
    
    plt.subplot(2, 6, 2) 
#     plt.hist(pickr[:,0,1],bins=20,color=colors[0],label=labels[0],range=(0,0.3))
    plt.hist(pickr[:,1,1],bins=20,color=colors[1],label=labels[1],range=(0,0.3))
#     plt.legend(labels)
    plt.title("Picks count")
    plt.xlabel("Probability")
    plt.ylim(0,20)
    plt.xlim(0.05,0.3)
    
    plt.subplot(2, 6, 7) 
    count=0
    for ch in range(image.shape[0]):
        if pickr[ch,1,1] > thr and pickr[ch,1,0]>1:
            plt.plot(tax, image[ch])
            count=count+1
            
    plt.subplot(2, 6, 8) 
    snr=0
    for ch in range(image.shape[0]):
        if pickr[ch,1,1] > thr and pickr[ch,1,0]>1:
            pt=int(pickr[ch,1,0]*25)
            plt.plot(np.arange(len(image[ch, pt-25:pt+25]))/25,image[ch, pt-25:pt+25])
            snr+=np.std(image[ch, pt:pt+25]) / (np.std(image[ch, pt-25:pt])+1e-7)
    snr=snr/(count+1e-7)
            
    plt.title(str(count) + " picks totally | SNR "+str(round(snr,1)))
    

    # oneDENOISED  
    image = oneDenoise_1[i,::ch_itv,:]
    x, y = np.arange(image.shape[1])/25, np.arange(image.shape[0])*ch_itv*dchan/1000
    plt.subplot(2, 6, 3) 
    plt.pcolormesh(x, y, image, shading='auto', vmin=-bound, vmax=bound, cmap=cmap)
#     plt.scatter(pick1[:,0,0],y, s=12,marker='o',c="b",alpha=0.3)
    plt.scatter(pick1[:,1,0],y, s=12,marker='o',c=colors[1],alpha=0.3)
    plt.title("Ensemble Picking on Denoised data")
    plt.xlabel("Time since starttime (s)")
    plt.ylabel("Distance along cable (km)")

    plt.subplot(2, 6, 4) 
#     plt.hist(pick1[:,0,1],bins=20,color=colors[0],label=labels[0],range=(0,0.3))
    plt.hist(pick1[:,1,1],bins=20,color=colors[1],label=labels[1],range=(0,0.3))
#     plt.legend(labels)
    plt.title("Picks count")
    plt.xlabel("Probability")
    plt.ylim(0,20)
    plt.xlim(0.05,0.3)
    
    plt.subplot(2, 6, 9) 
    count=0
    for ch in range(image.shape[0]):
        if pick1[ch,1,1] > thr and pick1[ch,1,0]>1:
            plt.plot(tax, image[ch])
            count=count+1
            
    plt.subplot(2, 6, 10) 
    for ch in range(image.shape[0]):
        if pick1[ch,1,1] > thr and pick1[ch,1,0]>1:
            pt=int(pick1[ch,1,0]*25)
            plt.plot(np.arange(len(image[ch, pt-25:pt+25]))/25,image[ch, pt-25:pt+25])
            snr+=np.std(image[ch, pt:pt+25]) / (np.std(image[ch, pt-25:pt])+1e-7)
    snr=snr/(count+1e-7)
            
    plt.title(str(count) + " picks totally | SNR "+str(round(snr,1)))
    
    # mulDENOISED  
    image = mulDenoise_1[i,::ch_itv,:]
    x, y = np.arange(image.shape[1])/25, np.arange(image.shape[0])*ch_itv*dchan/1000
    plt.subplot(2, 6, 5) 
    plt.pcolormesh(x, y, image, shading='auto', vmin=-bound, vmax=bound, cmap=cmap)
#     plt.scatter(pick2[:,0,0],y, s=12,marker='o',c="b",alpha=0.3)
    plt.scatter(pick2[:,1,0],y, s=12,marker='o',c=colors[1],alpha=0.3)
    plt.title("Ensemble Picking on Denoised data")
    plt.xlabel("Time since starttime (s)")
    plt.ylabel("Distance along cable (km)")

    plt.subplot(2, 6, 6) 
#     plt.hist(pick2[:,0,1],bins=20,color=colors[0],label=labels[0],range=(0,0.3))
    plt.hist(pick2[:,1,1],bins=20,color=colors[1],label=labels[1],range=(0,0.3))
#     plt.legend(labels)
    plt.title("Picks count")
    plt.xlabel("Probability")
    plt.ylim(0,20)
    plt.xlim(0.05,0.3)
    
    plt.subplot(2, 6, 11) 
    count=0
    for ch in range(image.shape[0]):
        if pick2[ch,1,1] > thr and pick2[ch,1,0]>1:
            plt.plot(tax, image[ch])
            count=count+1
    
    plt.subplot(2, 6, 12) 
    for ch in range(image.shape[0]):
        if pick2[ch,1,1] > thr and pick2[ch,1,0]>1:
            pt=int(pick2[ch,1,0]*25)
            plt.plot(np.arange(len(image[ch, pt-25:pt+25]))/25,image[ch, pt-25:pt+25])
            snr+=np.std(image[ch, pt:pt+25]) / (np.std(image[ch, pt-25:pt])+1e-7)
    snr=snr/(count+1e-7)
            
    plt.title(str(count) + " picks totally | SNR "+str(round(snr,1)))
            
    plt.savefig(f"./plots/ELEP_bb_picks_event_smaller_{i}.png")